# Algorithm for Protein Folding in 2D Lattice

In [1]:
import numpy as np 

### Step 1 - Initialize Grid
- Create a 2D grid or lattice to represent the folding environment.
- Place the hydrophobic (H) and polar (P) amino acids randomly on the grid.

In [6]:
grid_size = 5
num_hydrophobic = 5
num_polar = 5

def initialize_grid(grid_size, num_hydrophobic, num_polar):
    grid = np.zeros((grid_size, grid_size), dtype=str)
    
    # Place hydrophobic amino acids (H) randomly on the grid
    hydrophobic_indices = np.random.choice(grid_size ** 2, num_hydrophobic, replace=False)
    grid.ravel()[hydrophobic_indices] = 'H'
    
    # Place polar amino acids (P) randomly on the grid
    polar_indices = np.random.choice(np.where(grid == '')[0], num_polar, replace=False)
    grid.ravel()[polar_indices] = 'P'
    
    return grid

protein_grid = initialize_grid(grid_size, num_hydrophobic, num_polar)
protein_grid

array([['P', 'P', 'P', 'P', ''],
       ['', '', '', 'H', ''],
       ['', '', '', '', ''],
       ['H', '', '', 'H', ''],
       ['', 'H', '', 'H', '']], dtype='<U1')

### Step 2 - Calculate Initial Energy

- Calculate the initial energy of the protein based on the arrangement of H-amino acids (adjacent and non-adjacent) and P-amino acids.

In [7]:
def calculate_initial_energy(grid):
    energy = 0
    
    # Define neighboring offsets (up, down, left, right)
    offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    # Iterate over each cell in the grid
    for row in range(grid.shape[0]):
        for col in range(grid.shape[1]):
            if grid[row, col] == 'H':
                # Calculate interactions with neighboring hydrophobic amino acids
                for offset_row, offset_col in offsets:
                    new_row, new_col = row + offset_row, col + offset_col
                    if 0 <= new_row < grid.shape[0] and 0 <= new_col < grid.shape[1]:
                        if grid[new_row, new_col] == 'H':
                            energy -= 1  # Adjacent hydrophobic amino acids contribute -1 energy
            
            elif grid[row, col] == 'P':
                # Calculate interactions with neighboring hydrophobic amino acids
                for offset_row, offset_col in offsets:
                    new_row, new_col = row + offset_row, col + offset_col
                    if 0 <= new_row < grid.shape[0] and 0 <= new_col < grid.shape[1]:
                        if grid[new_row, new_col] == 'H':
                            energy += 0  # No interaction with polar-amino acid neighbors
                        
    return energy


# Calculate initial energy
initial_energy = calculate_initial_energy(protein_grid)
print("Initial Energy:", initial_energy)

Initial Energy: -2


### Step 3 - Select Amino Acid to Move
- Choose a random amino acid from the grid. Consider both hydrophobic and polar amino acids.

### Step 4 - Propose Move

- Propose moving the selected amino acid to a neighboring position on the grid.
- Calculate the change in energy due to this move.

### Step 5 - Accept or Reject Move
- If the proposed move reduces the energy (makes the protein more stable), accept the move with a probability based on the energy change.
- If the move increases the energy, accept it with a lower probability. This introduces randomness and helps avoid getting stuck in local energy minima.

### Step 6 - Update energy
- Update the energy of the protein after accepting or rejecting the move.

In [9]:
def select_amino_acid_to_move(grid):
    # Get the indices of all amino acids (H and P) on the grid
    amino_acid_indices = np.transpose(np.where(grid != ''))
    
    # Randomly select an index from the amino acid indices
    selected_index = np.random.choice(amino_acid_indices.shape[0])
    selected_row, selected_col = amino_acid_indices[selected_index]
    
    return selected_row, selected_col

def propose_move(selected_row, selected_col, grid):
    # Choose a random neighboring position for the proposed move
    offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    offset_row, offset_col = offsets[np.random.choice(len(offsets))]
    
    new_row, new_col = selected_row + offset_row, selected_col + offset_col
    
    return new_row, new_col

def calculate_energy_change(initial_energy, proposed_row, proposed_col, grid):
    # Calculate energy change based on interactions with neighbors
    # Similar to the energy calculation in step 2
    
    return energy_change

def accept_or_reject_move(initial_energy, proposed_row, proposed_col, grid):
    # Calculate the energy change due to the proposed move
    energy_change = calculate_energy_change(initial_energy, proposed_row, proposed_col, grid)
    
    # Accept the move with a probability based on the energy change
    if energy_change <= 0:
        return True  # Accept the move
    else:
        probability = np.exp(-energy_change)
        return np.random.rand() < probability

def update_energy(initial_energy, proposed_row, proposed_col, grid):
    # Calculate the energy change due to the proposed move
    energy_change = calculate_energy_change(initial_energy, proposed_row, proposed_col, grid)
    
    # Update the energy by adding the energy change
    return initial_energy + energy_change


# Set parameters
grid_size = 10
num_hydrophobic = 5
num_polar = 5

# Initialize the grid (using the previous example)
protein_grid = initialize_grid(grid_size, num_hydrophobic, num_polar)

# Select an amino acid to move
selected_row, selected_col = select_amino_acid_to_move(protein_grid)

# Propose a move for the selected amino acid
proposed_row, proposed_col = propose_move(selected_row, selected_col, protein_grid)

# Calculate initial energy
initial_energy = calculate_initial_energy(protein_grid)

# Accept or reject the proposed move
move_accepted = accept_or_reject_move(initial_energy, proposed_row, proposed_col, protein_grid)

if move_accepted:
    # Update the grid and energy after accepting the move
    protein_grid[selected_row, selected_col] = ''
    protein_grid[proposed_row, proposed_col] = 'H'  # For simplicity, assuming moving to a hydrophobic site
    updated_energy = update_energy(initial_energy, proposed_row, proposed_col, protein_grid)
    print("Move accepted. Updated Energy:", updated_energy)
else:
    print("Move rejected.")


NameError: name 'energy_change' is not defined

### Step 7 - Perform folding
In this code, the perform_folding function uses a while loop to iteratively perform steps 3 to 6 for a maximum number of iterations. It also includes a convergence check that monitors energy changes over iterations. If the energy change becomes very small (close to convergence), the folding process is considered complete, and the loop breaks.

In [45]:
def perform_folding(grid, max_iterations):
    # Initialize energy and iteration counter
    current_energy = calculate_initial_energy(grid)
    iteration = 0
    
    while iteration < max_iterations:
        # Select an amino acid to move
        selected_row, selected_col = select_amino_acid_to_move(grid)
        
        # Propose a move for the selected amino acid
        proposed_row, proposed_col = propose_move(selected_row, selected_col, grid)
        
        # Calculate initial energy
        initial_energy = current_energy
        
        # Accept or reject the proposed move
        move_accepted = accept_or_reject_move(initial_energy, proposed_row, proposed_col, grid)
        
        if move_accepted:
            # Update the grid and energy after accepting the move
            grid[selected_row, selected_col] = ''
            grid[proposed_row, proposed_col] = 'H'  # For simplicity, assuming moving to a hydrophobic site
            current_energy = update_energy(initial_energy, proposed_row, proposed_col, grid)
        
        # Check for energy convergence
        if iteration > 0 and np.isclose(current_energy, previous_energy, atol=1e-6):
            print(f"Energy converged at iteration {iteration}. Energy: {current_energy}")
            break
        
        # Update previous energy for convergence check
        previous_energy = current_energy
        
        iteration += 1
    
    print("Folding process complete.")
    return grid, current_energy

# Set parameters
grid_size = 10
num_hydrophobic = 5
num_polar = 5
max_iterations = 10 # Maximum number of iterations

# Initialize the grid (using the previous example)
protein_grid = initialize_grid(grid_size, num_hydrophobic, num_polar)

# Perform folding
final_grid, final_energy = perform_folding(protein_grid, max_iterations)

NameError: name 'energy_change' is not defined

In [ ]:
if move_accepted:
    # Update the grid and energy after accepting the move
    protein_grid[selected_row, selected_col] = ''
    protein_grid[proposed_row, proposed_col] = 'H'  # For simplicity, assuming moving to a hydrophobic site
    updated_energy = update_energy(initial_energy, proposed_row, proposed_col, protein_grid)
    print("Move accepted. Updated Energy:", updated_energy)
else:
    print("Move rejected.")